# Recitation 2: Regression & Time Series
The following function will be useful.

In [ ]:
# Calculate the Mean Absolute Percentage Error (MAPE) for training and testing data
def calculate_mape(y_true, y_pred):
    return (100 * abs((y_true - y_pred) / y_true)).mean()

In [ ]:
def mape_error(actual, forecast):
    """
    Compute the Mean Absolute Percentage Error (MAPE)
    
    Parameters:
    - actual: list of actual values
    - forecast: list of forecasted values
    
    Returns:
    - MAPE: Mean Absolute Percentage Error
    """
    n = len(actual)
    if n == 0 or len(forecast) != n:
        raise ValueError("Input lists must be of same non-zero length")
    
    mape = sum(abs((a - f) / a) for a, f in zip(actual, forecast)) / n * 100
    return mape

Before running this notebook, check that every package that we will use is installed

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from pmdarima import auto_arima
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf
import warnings
warnings.filterwarnings('ignore')

# If a package is missing, install it by writing !pip install package_name

In [ ]:
df = pd.read_csv("boston_housing.csv")

In [ ]:
df

In [ ]:
df.shape

In the Boston Housing dataset, the target variable is:

- MEDV: Median value of owner-occupied homes in $1000s.

This is the variable that we typically try to predict based on the other features in the dataset when performing regression analyses. Essentially, it represents the median price of homes in various areas of Boston.

In [ ]:
# Using the 'RM' feature (average number of rooms per dwelling) to predict 'MEDV' (Median value of owner-occupied homes)
X = df['RM']

In [ ]:
y = boston.target

In [ ]:
# Adding a constant (for intercept)
X = sm.add_constant(X)

In [ ]:
# Selecting some indices from the dataset for the analysis
X_sample = X.iloc[0:400]
y_sample = y[0:400]

In [ ]:
# Fitting the regression model
model = sm.OLS(y_sample, X_sample).fit()

In [ ]:
# Extracting model results
slope_estimate = model.params['RM']
p_value = model.pvalues['RM']
r_squared = model.rsquared

# Plotting Fitted Values vs. Residuals
fitted_values = model.fittedvalues
residuals = model.resid

plt.figure(figsize=(8, 6))
plt.scatter(fitted_values, residuals, alpha=0.6)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values - Example')
plt.ylabel('Residuals')
plt.title('Fitted Values vs. Residuals')
plt.grid(True)
plt.show()

slope_estimate, p_value, r_squared

- X-axis (Fitted Values): These are the predicted values of the dependent variable (in this case, the median home value, MEDV) based on the regression model.

- Y-axis (Residuals): The residuals represent the difference between the observed values (actual data) and the fitted values (predictions). Specifically:

Residual= Observed Value− Fitted Value

- Horizontal Line at Residuals = 0: This line signifies where the prediction matches the actual value perfectly. Residuals above the line indicate under-prediction by the model, while residuals below the line indicate over-prediction.

The purpose of the Fitted Values vs. Residuals plot is to check the assumptions of the linear regression model, particularly:

1. Linearity: The relationship between the predictors and the response should be linear.
2. Independence: The residuals are independent.
3. Homoscedasticity: The variance of the residuals is constant across all levels of the independent variables.
4. Normality: The residuals should be approximately normally distributed.
For a well-fitting model:

The residuals should be randomly scattered around the horizontal line at 0.
There shouldn't be any clear pattern or trend in the residuals.
If we see patterns in this plot, it may indicate potential problems with the model, such as non-linearity, non-constant variance (heteroscedasticity), or outliers.

**In this plot** the residuals seem to be reasonably scattered around the horizontal line at 0. This suggests that the model's errors are random, which is a good sign.
There's no clear funnel shape, which would indicate heteroscedasticity (non-constant variance of errors). This is also positive.
There doesn't appear to be any distinct curve or pattern in the residuals, suggesting the linearity assumption might hold.
However, there are a few considerations:

Some points are far from the central cluster, hinting at potential outliers or high-leverage points.
A more thorough analysis would involve other diagnostic plots and tests, such as a quantile-quantile (Q-Q) plot to check the normality of residuals or leverage vs. residual plots to identify influential points.
Overall, based solely on the Fitted Values vs. Residuals plot, the model seems reasonably good. The residuals appear random and without any clear patterns. However, a comprehensive evaluation of a regression model would involve examining multiple diagnostic plots and statistics to ensure all assumptions are met and that the model is robust.

In [ ]:
model.summary()

1. R-squared and Adjusted R-squared: **R-squared: 0.567**. This indicates that approximately 56.7% of the variability in the median home values can be explained by the average number of rooms per dwelling in this subset of the data. Adjusted R-squared: 0.566. It's very close to the R-squared value, indicating that most of the included predictors (in this case, just 'RM') are relevant to the model.
2. F-statistic and its associated p-value: The F-statistic tests the hypothesis that all regression coefficients are equal to zero (meaning the model fits no better than a horizontal line or a model with no predictors). A significant F-statistic (in this case, very close to zero) indicates that the predictors are valuable.
3. Coefficients and their Significance: Intercept (const): -35.2609. This is the predicted median home value when RM is zero (though it doesn't have a practical interpretation since a dwelling can't have zero rooms). **RM: 9.4055**. This indicates that for every additional room, the median value of homes increases by about \$9,406, holding other factors constant. Both coefficients have p-values very close to zero, indicating that they are statistically significant.

In [ ]:
# Plotting the regression results on the data
plt.figure(figsize=(10, 7))

# Scatter plot of the data
plt.scatter(X_sample['RM'], y_sample, label='Data', alpha=0.6)

# Regression line
plt.plot(X_sample['RM'], fitted_values, color='red', label='Regression Line')

plt.title('Regression of Median Home Value on Number of Rooms')
plt.xlabel('Average Number of Rooms per Dwelling (RM)')
plt.ylabel('Median Value of Owner-occupied Homes (in $1000s)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Predicting values using the model
predicted_values_multivariate = model.predict(X_sample)

# Calculating the error metrics
MAE = mean_absolute_error(y_sample, predicted_values_multivariate)
RMSE = mean_squared_error(y_sample, predicted_values_multivariate, squared=False)
MAPE = np.mean(np.abs((y_sample - predicted_values_multivariate) / y_sample)) * 100
R_squared = model.rsquared
print("MAE", MAE, "\nRMSE", RMSE, "\nMAPE", MAPE,"\nR2", R_squared)

The MAE, RMSE, and MAPE provide insights into the model's prediction accuracy. Low values of MAE and RMSE, along with the high R-squared suggest that a model provides a good fit to the data and makes reasonably accurate predictions. 

However, always consider the context and the domain when interpreting these metrics. For instance, in a scenario where housing prices range widely, an MAE of $1867.60 might be considered very good. However, in a market with more consistent and narrow-ranging prices, the same error might be seen as less acceptable.

### Multiple Linear Regression
We do the same but with $X = ['RM', 'LSTAT', 'PTRATIO']$ as predictors.

In [ ]:
# Using multiple features to predict 'MEDV' (Median value of owner-occupied homes)
# We'll use 'RM' (average number of rooms), 'LSTAT' (% lower status of the population), and 'PTRATIO' (pupil-teacher ratio by town) for this example
features = ['RM', 'LSTAT', 'PTRATIO']
X_multivariate = df[features]

# Adding a constant (for intercept)
X_multivariate = sm.add_constant(X_multivariate)

# Selecting the same indices from our previous sample for consistency
X_multivariate_sample = X_multivariate.iloc[1:400]
y_multivariate_sample = y[1:400]

# Fitting the multivariate regression model using the first 400 data points
model_multivariate = sm.OLS(y_multivariate_sample, X_multivariate_sample).fit()

# 1. Fitted Values vs. Residuals
fitted_values_multivariate = model_multivariate.fittedvalues
residuals_multivariate = model_multivariate.resid

# 2. Q-Q Plot
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

# Plotting Fitted Values vs. Residuals
axes.scatter(fitted_values_multivariate, residuals_multivariate, alpha=0.6)
axes.axhline(0, color='red', linestyle='--')
axes.set_title('Fitted Values vs. Residuals')
axes.set_xlabel('Fitted Values')
axes.set_ylabel('Residuals')
axes.grid(True)


plt.tight_layout()
plt.show()

In [ ]:
# Getting the predicted values
predicted_values = model_multivariate.fittedvalues

# Plotting with appropriate labels
plt.figure(figsize=(10, 7))
plt.scatter(y_multivariate_sample, predicted_values, alpha=0.6, label='Data Points')
plt.plot([min(y_multivariate_sample), max(y_multivariate_sample)], 
         [min(y_multivariate_sample), max(y_multivariate_sample)], 
         color='red', linestyle='--', label='Perfect Prediction Line')
plt.xlabel('Actual Median Home Values ($1000s)')
plt.ylabel('Predicted Median Home Values ($1000s)')
plt.title('Actual vs. Predicted Home Values')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model_multivariate.summary()

##### **R-squared and Adjusted R-squared**:
- **R-squared**: 0.671. This metric denotes that approximately 67.1% of the variability in the median home values can be accounted for by our model using the three predictor variables.
- **Adjusted R-squared**: 0.668. A slight decrease from the R-squared value, which is expected as it adjusts for the number of predictors in the model. This close value indicates the predictors are relevant.

##### **F-statistic and its associated p-value**:
- The **F-statistic** tests the overall significance of the model. With a p-value close to zero, this demonstrates that at least one of the predictors is significant.

##### **Coefficients and their Significance**:
- **Intercept (const)**: 5.5940. This is the median home value when all predictors are zero. However, this doesn't necessarily have a direct practical meaning.
- **RM**: 5.7950. For every additional room, the median value of homes is predicted to increase by about \$5,795, holding other variables constant.
- **LSTAT**: -0.4784. For every 1\% increase in the lower status of the population, the median value of homes is predicted to decrease by about \$478.
- **PTRATIO**: -0.6986. For every unit increase in the pupil-teacher ratio, the median value of homes is predicted to decrease by about $698.
- All predictors have p-values very close to zero, signifying their statistical significance.


In [ ]:
# Predicting values using the model
predicted_values_multivariate = model_multivariate.predict(X_multivariate_sample)

# Calculating the error metrics
MAE = mean_absolute_error(y_multivariate_sample, predicted_values_multivariate)
RMSE = mean_squared_error(y_multivariate_sample, predicted_values_multivariate, squared=False)
MAPE = calculate_mape(y_multivariate_sample, predicted_values_multivariate)
R_squared = model_multivariate.rsquared
MAE, RMSE, MAPE, R_squared

The MAE, RMSE, and MAPE provide insights into the model's prediction accuracy. The relatively low values of MAE and RMSE, along with the high R-squared, suggest that the model provides a good fit to the data and makes reasonably accurate predictions. However, always consider the context and the domain when interpreting these metrics. For instance, in a scenario where housing prices range widely, an MAE of $1867.60 might be considered very good. However, in a market with more consistent and narrow-ranging prices, the same error might be seen as less acceptable.

# Time Series Analysis

Let's read in some data on Monthly Average Temperature in Alaska.


Source: https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data

In [ ]:
# Read the data
df = pd.read_csv("Alaska_AvgTemp.csv")
print(df.head())
print(df['Date'].dtype)
df

Whenever we are working with dates, it is better to transform them in the 'datetime' format.

In [ ]:
# Convert 'Date' column to datetime type
df['Date'] = pd.to_datetime(df['Date'])
print(df['Date'].dtype)

Now we visualize the time series.

In [ ]:
# Set up the plot style and size
plt.figure(figsize=(5, 7))

# Plot the data
plt.plot(df['Date'], df['AverageTemperature'], color='dodgerblue', linewidth=2, label='Average Temperature')
plt.fill_between(df['Date'], df['AverageTemperature'], color='dodgerblue', alpha=0.1)

# Set the title and labels
plt.title('Monthly Average Temperature in Alaska (1950 onwards)', fontsize=18)
plt.xlabel('Date', fontsize=14)
plt.ylabel('Average Temperature (°C)', fontsize=14)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()

# Display the plot
plt.tight_layout()
plt.show()

We will transform the 'data' Dataframe into a Series with monthly frequency ('MS')

In [ ]:
df['AverageTemperature']

In [ ]:
# Create time series data with monthly frequency
temp = pd.Series(df['AverageTemperature'].values, index=df['Date'])
temp.index.freq = 'MS'  # Monthly start frequency

# Decomposing the time series
decomp = seasonal_decompose(temp)

In [ ]:
# Setting up the figure and axes
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

# Observed
axes[0].plot(decomp.observed, color='dodgerblue')
axes[0].set_title('Observed')
axes[0].grid(True, which='both', linestyle='--', linewidth=0.5)

# Trend
axes[1].plot(decomp.trend, color='dodgerblue')
axes[1].set_title('Trend')
axes[1].grid(True, which='both', linestyle='--', linewidth=0.5)

# Seasonal
axes[2].plot(decomp.seasonal, color='dodgerblue')
axes[2].set_title('Seasonal')
axes[2].grid(True, which='both', linestyle='--', linewidth=0.5)

# Residual as a line plot
axes[3].plot(decomp.resid, color='dodgerblue')
axes[3].set_title('Residual')
axes[3].grid(True, which='both', linestyle='--', linewidth=0.5)

# Adjusting the layout
plt.tight_layout()
plt.show()

For time series data, we usually split our training and test sets by date (before/after a certain time)

In [ ]:
# Splitting the data
train = df[df['Date'] <= "2000-01-01"]
test = df[df['Date'] > "2000-01-01"]
train.head()

In [ ]:
test.head()

The `ARIMA()` function from the `statsmodels.tsa.arima.model` package let's use fit an ARIMA model with order = (p, d, q). Let's create a time series object of the training set

In [ ]:
# ARIMA model
temp_train = pd.Series(train['AverageTemperature'].values, index=train['Date'])

# 'MS' stands for "Month Start frequency". It indicates that each data point represents the start of a month.
temp_train.index.freq = 'MS'

mod1 = ARIMA(temp_train, order=(2, 1, 1))
results1 = mod1.fit()
print(results1.summary())

# Accuracy
print(results1.aic)

In [ ]:
# Forecasting
forecast1 = results1.forecast(steps=4)
print(forecast1)

The `auto_arima()` function does automatic model selection for us, and computes the "best-fit" (p, d, q)(P, D, Q).

Examples in the choice of $m$:
- Daily data with weekly patterns: $m=7$.
- Monthly data with yearly patterns: $m=12$.
- Hourly data with daily patterns: $m=24$ (assuming patterns repeat every day).

In [ ]:
temp_train

In [ ]:
# Auto ARIMA
mod2 = auto_arima(temp_train, seasonal=True, m=12, trace=True, error_action='ignore', suppress_warnings=True)

In [ ]:
print(mod2.summary())

In [ ]:
# Extract the parameters
p, d, q = mod2.order
P, D, Q, s = mod2.seasonal_order

print(f"The ARIMA parameters are p = {p}, d = {d}, q = {q}")
print(f"The seasonal ARIMA parameters are P = {P}, D = {D}, Q = {Q}")

In [ ]:
# Fitted values and residuals
fitted = mod2.predict_in_sample()
residuals = temp_train - fitted

# Predictions
forecast2 = mod2.predict(n_periods=163)
train_pred = fitted
test_pred = forecast2

In [ ]:
# Calculate MAPE
train_mape = calculate_mape(train['AverageTemperature'].values, train_pred)
test_mape = calculate_mape(test['AverageTemperature'].values, test_pred)
print("Training MAPE:", train_mape)
print("Test MAPE:", test_mape)

In [ ]:
forecast2